# Importing model and necessary packages 

In [20]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [28]:
model_path ='C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/detect.tflite'
label_path='C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/labelmap.txt'

In [23]:
# Load the Tensorflow Lite model into memory
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

float_input = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5


In [24]:

# Import required packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt


# Defining a function to test on images

In [25]:
# Import required packages
import os
import cv2
import numpy as np
import glob
import random
import importlib.util
import matplotlib.pyplot as plt

# Define function for object detection using a TensorFlow Lite model and displaying results
def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/path', txt_only=False):
    """
    Perform object detection using a TensorFlow Lite model on test images.

    Parameters:
        modelpath (str): Path to the TensorFlow Lite model file.
        imgpath (str): Path to the folder containing test images.
        lblpath (str): Path to the label map file.
        min_conf (float, optional): Minimum confidence threshold for displaying detections. Default is 0.5.
        num_test_images (int, optional): Number of test images to randomly select for inference. Default is 10.
        savepath (str, optional): Path to the folder where the text files will be saved. Default is '/content/results'.
        txt_only (bool, optional): Flag to control whether to display images with results or just save the results in text files. Default is False.

    Returns:
        None.
    """

    # Grab filenames of all images in test folder
    images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

    # Load the label map into memory
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the Tensorflow Lite model into memory
    interpreter = tf.lite.Interpreter(model_path=modelpath)
    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    float_input = (input_details[0]['dtype'] == np.float32)

    input_mean = 127.5
    input_std = 127.5

    # Randomly select test images
    images_to_test = random.sample(images, num_test_images)

    # Loop over every image and perform detection
    for image_path in images_to_test:

        # Load image and resize to expected shape [1xHxWx3]
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        imH, imW, _ = image.shape
        image_resized = cv2.resize(image_rgb, (width, height))
        input_data = np.expand_dims(image_resized, axis=0)

        # Normalize pixel values if using a floating model (i.e., if model is non-quantized)
        if float_input:
            input_data = (np.float32(input_data) - input_mean) / input_std

        # Perform the actual detection by running the model with the image as input
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        # Retrieve detection results
        boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # Bounding box coordinates of detected objects
        classes = interpreter.get_tensor(output_details[3]['index'])[0]  # Class index of detected objects
        scores = interpreter.get_tensor(output_details[0]['index'])[0]  # Confidence of detected objects

        detections = []

        # Loop over all detections and draw detection box if confidence is above minimum threshold
        for i in range(len(scores)):
            if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

                # Get bounding box coordinates and draw box
                # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
                ymin = int(max(1, (boxes[i][0] * imH)))
                xmin = int(max(1, (boxes[i][1] * imW)))
                ymax = int(min(imH, (boxes[i][2] * imH)))
                xmax = int(min(imW, (boxes[i][3] * imW)))

                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

                # Draw label
                object_name = labels[int(classes[i])]  # Look up object name from "labels" array using class index
                label = '%s: %d%%' % (object_name, int(scores[i] * 100))  # Example: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)  # Get font size
                label_ymin = max(ymin, labelSize[1] + 10)  # Make sure not to draw label too close to the top of the window
                cv2.rectangle(image, (xmin, label_ymin - labelSize[1] - 10), (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED)  # Draw white box to put label text in
                cv2.putText(image, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)  # Draw label text

                detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

        # All the results have been drawn on the image, now display the image or save detection results in .txt files (for calculating mAP)
        if not txt_only:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(12, 16))
            plt.imshow(image)
            plt.show()
        else:
            # Get filenames and paths
            image_fn = os.path.basename(image_path)
            base_fn, ext = os.path.splitext(image_fn)
            txt_result_fn = base_fn + '.txt'
            txt_savepath = os.path.join(savepath, txt_result_fn)

            # Write results to text file
            # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
            with open(txt_savepath, 'w') as f:
                for detection in detections:
                  f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

    return


# Defining a function to run inference with webcam

In [12]:
!pip install --upgrade --user opencv-python



  Using cached opencv_python-4.8.0.74-cp37-abi3-win_amd64.whl (38.1 MB)


In [29]:
import cv2
import numpy as np
import tensorflow as tf
import PIL.Image
from IPython.display import display, clear_output

def tflite_detect_camera(modelpath, lblpath, min_conf=0.5):
    """
    Perform real-time object detection using a TensorFlow Lite model with the webcam.

    Parameters:
        modelpath (str): Path to the TensorFlow Lite model file.
        lblpath (str): Path to the label map file.
        min_conf (float, optional): Minimum confidence threshold for displaying detections. Default is 0.5.

    Returns:
        None.
    """

    # Load the label map into memory
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the TensorFlow Lite model into memory
    interpreter = tf.lite.Interpreter(model_path=modelpath)
    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    float_input = (input_details[0]['dtype'] == np.float32)

    input_mean = 127.5
    input_std = 127.5

    # Initialize webcam stream
    cap = cv2.VideoCapture(0)  # Use 0 for the default camera or a specific camera index

    try:
        while True:
            # Capture a frame from the webcam
            ret, frame = cap.read()

            # Check if the frame is captured successfully
            if not ret:
                print("Error: Could not capture frame.")
                break

            # Preprocess the input frame (resize and normalize)
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_resized = cv2.resize(image_rgb, (width, height))
            input_data = np.expand_dims(image_resized, axis=0)

            # Normalize pixel values if using a floating model (i.e., if model is non-quantized)
            if float_input:
                input_data = (np.float32(input_data) - input_mean) / input_std

            # Perform object detection by running the model with the image as input
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()

            # Retrieve detection results
            boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # Bounding box coordinates of detected objects
            classes = interpreter.get_tensor(output_details[3]['index'])[0]  # Class index of detected objects
            scores = interpreter.get_tensor(output_details[0]['index'])[0]  # Confidence of detected objects

            # Loop over all detections and draw detection box if confidence is above minimum threshold
            for i in range(len(scores)):
                if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
                    # Get bounding box coordinates and draw box
                    ymin = int(max(1, (boxes[i][0] * frame.shape[0])))
                    xmin = int(max(1, (boxes[i][1] * frame.shape[1])))
                    ymax = int(min(frame.shape[0], (boxes[i][2] * frame.shape[0])))
                    xmax = int(min(frame.shape[1], (boxes[i][3] * frame.shape[1])))

                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

                    # Draw label
                    object_name = labels[int(classes[i])]
                    label = '%s: %.2f' % (object_name, scores[i])
                    cv2.putText(frame, label, (xmin, ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Convert the frame to RGB and display it using IPython.display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = PIL.Image.fromarray(frame_rgb)
            display(pil_image)

            # Clear the previous output to show the updated frame
            clear_output(wait=True)

    except KeyboardInterrupt:
        # Press 'Ctrl+C' to stop the webcam stream and close the window
        print("Webcam stream stopped by user.")

    # Release the webcam and close the OpenCV window
    cap.release()
    clear_output()


In [30]:
tflite_detect_camera(model_path, label_path)
#if you want to stop the stream, select the cell block, go to kernel then interrupt.

# Defining a function that saves the detected images

the following code adds cropping the ROI(region of interest) from the detected licence plates and saves them in a local folder

In [31]:
import cv2
import numpy as np
import tensorflow as tf
import PIL.Image
import os

def tflite_detect_camera_ROI(modelpath, lblpath, min_conf=0.5, save_folder="C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates"):
    """
    Perform real-time object detection using a TensorFlow Lite model with the webcam.

    Parameters:
        modelpath (str): Path to the TensorFlow Lite model file.
        lblpath (str): Path to the label map file.
        min_conf (float, optional): Minimum confidence threshold for displaying detections. Default is 0.5.
        save_folder (str, optional): Path to save the cropped license plate images. Default is "detected_plates/".

    Returns:
        None.
    """
    
    # Create the folder to save the detected license plate images
    #os.makedirs(save_folder, exist_ok=True)

    # Load the label map into memory
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the TensorFlow Lite model into memory
    interpreter = tf.lite.Interpreter(model_path=modelpath)
    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    float_input = (input_details[0]['dtype'] == np.float32)

    input_mean = 127.5
    input_std = 127.5

    # Initialize webcam stream
    cap = cv2.VideoCapture(0)  # Use 0 for the default camera or a specific camera index

    # Create a counter variable to ensure unique image names
    counter = 0

    try:
        while True:
            # Capture a frame from the webcam
            ret, frame = cap.read()

            # Check if the frame is captured successfully
            if not ret:
                print("Error: Could not capture frame.")
                break

            # Preprocess the input frame (resize and normalize)
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image_resized = cv2.resize(image_rgb, (width, height))
            input_data = np.expand_dims(image_resized, axis=0)

            # Normalize pixel values if using a floating model (i.e., if model is non-quantized)
            if float_input:
                input_data = (np.float32(input_data) - input_mean) / input_std

            # Perform object detection by running the model with the image as input
            interpreter.set_tensor(input_details[0]['index'], input_data)
            interpreter.invoke()

            # Retrieve detection results
            boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # Bounding box coordinates of detected objects
            classes = interpreter.get_tensor(output_details[3]['index'])[0]  # Class index of detected objects
            scores = interpreter.get_tensor(output_details[0]['index'])[0]  # Confidence of detected objects

            # Loop over all detections and draw detection box if confidence is above minimum threshold
            for i in range(len(scores)):
                if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
                    # Get bounding box coordinates and draw box
                    ymin = int(max(1, (boxes[i][0] * frame.shape[0])))
                    xmin = int(max(1, (boxes[i][1] * frame.shape[1])))
                    ymax = int(min(frame.shape[0], (boxes[i][2] * frame.shape[0])))
                    xmax = int(min(frame.shape[1], (boxes[i][3] * frame.shape[1])))

                    # Draw bounding box and label
                    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)
                    object_name = labels[int(classes[i])]
                    label = '%s: %.2f' % (object_name, scores[i])
                    cv2.putText(frame, label, (xmin, ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                    # Crop the detected license plate region
                    license_plate = frame[ymin:ymax, xmin:xmax]

                    # Save the cropped license plate as an image with a unique name using the counter
                    image_name = f"plate_{counter}.jpg"
                    counter += 1
                    save_filepath = os.path.join(save_folder, image_name)
                    cv2.imwrite(save_filepath, license_plate)

            # Convert the frame to RGB and display it using IPython.display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = PIL.Image.fromarray(frame_rgb)
            display(pil_image)

            # Clear the previous output to show the updated frame
            clear_output(wait=True)

    except KeyboardInterrupt:
        # Press 'Ctrl+C' to stop the webcam stream and close the window
        print("Webcam stream stopped by user.")

    # Release the webcam and close the OpenCV window
    cap.release()
    clear_output()


In [32]:
# Replace these paths with your specific model and label map paths
model_path = 'C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/detect.tflite'
label_path = 'C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/labelmap.txt'

# Set the minimum confidence threshold (you can adjust this value as needed)
min_confidence = 0.5
save_folder="C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates"

# Call the function for real-time license plate detection and cropping
tflite_detect_camera_ROI(model_path, label_path, min_confidence, save_folder)


# Appliying OCR to extract number plates from the saved images


In [6]:
pip install torch torchvision --upgrade --user


  Using cached torchvision-0.14.1-cp37-cp37m-win_amd64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [33]:
import torch




Testing on one image

In [34]:
import easyocr

def ocr_image(image_path, languages=['en']):
    """
    Perform OCR on an image and extract text.

    Parameters:
        image_path (str): Path to the input image.
        languages (list, optional): List of languages to recognize. Default is ['en'] (English).

    Returns:
        List of text detections in the format: [(bbox, text, score), ...]
    """
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(languages)

    # Read text from the image
    result = reader.readtext(image_path)

    return result

if __name__ == "__main__":
    # Replace 'path/to/your/image.jpg' with the actual path to the image you want to process
    image_path = "C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates/plate_0.jpg"
    
    # Perform OCR on the image
    detections = ocr_image(image_path)

    # Display the result
    for detection in detections:
        text = detection[1]
        print(text)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


AB |
FT 898


The following code defines a function that loops over the saved images, apply OCR on each image and saves the results in a csv file. The resulting csv file contains 2 columns: image name and detected licence plate

In [35]:
import os
import csv
import easyocr

def ocr_and_save_results(folder_path, output_csv="ocr_results.csv", languages=['en']):
    """
    Perform OCR on images in a folder and save the results to a CSV file.

    Parameters:
        folder_path (str): Path to the folder containing the detected plates.
        output_csv (str, optional): Path to save the CSV file. Default is "ocr_results.csv".
        languages (list, optional): List of languages to recognize. Default is ['en'] (English).

    Returns:
        None.
    """
    # Initialize the EasyOCR reader
    reader = easyocr.Reader(languages)

    # Create a CSV file to save the OCR results
    csv_file = os.path.join(folder_path, output_csv)

    # Open the CSV file in write mode and create a CSV writer
    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f)

        # Write the header row in the CSV file
        csv_writer.writerow(["Image Name", "Extracted Plate Number"])

        # Loop through the files in the folder
        for filename in os.listdir(folder_path):
            # Check if the file is an image (you can add more checks if needed)
            if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                # Construct the full path of the image
                image_path = os.path.join(folder_path, filename)

                # Perform OCR on the image
                detections = reader.readtext(image_path)

                # Extract the plate numbers from OCR results
                plate_numbers = [detection[1] for detection in detections]

                # Write the image name and extracted plate numbers to the CSV file
                csv_writer.writerow([filename, ', '.join(plate_numbers)])

    print(f"OCR results saved to {output_csv}.")

# Now you can call the function with the folder path and optional parameters
folder_path = "C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates"
ocr_and_save_results(folder_path, output_csv="ocr_results.csv", languages=['en'])


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


OCR results saved to ocr_results.csv.


Here, we establish a function aimed at refining our recurrent license plate detections, particularly when encountering multiple instances of the same plate. This function identifies and removes rows with duplicate license plate numbers obtained through Optical Character Recognition (OCR). Simultaneously, it eliminates recurring images from the folder dedicated to detected plates. 

In [ ]:
!pip install pandas


In [38]:
import os
import pandas as pd

def delete_recurrent_plates_and_images(csv_file, folder_path):
    """
    Delete rows with recurrent extracted plate numbers from the CSV file.
    Also, delete all images with recurring image names from the folder.

    Parameters:
        csv_file (str): Path to the CSV file containing OCR results.
        folder_path (str): Path to the folder containing the images.

    Returns:
        None.
    """
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Find recurrent extracted plate numbers
    recurrent_plate_numbers = df[df.duplicated(subset="Extracted Plate Number", keep=False)]["Extracted Plate Number"].unique()

    # Loop through the recurrent plate numbers
    for plate_number in recurrent_plate_numbers:
        # Get the image names corresponding to the recurrent plate number
        recurrent_images = df[df["Extracted Plate Number"] == plate_number]["Image Name"].tolist()

        # Loop through the images in the folder
        for filename in os.listdir(folder_path):
            # Check if the file is an image and has a recurrent image name
            if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')) and filename in recurrent_images:
                # Construct the full path of the image
                image_path = os.path.join(folder_path, filename)

                # Delete the image
                os.remove(image_path)

        # Remove all rows with the recurrent extracted plate number from the DataFrame
        df = df[df["Extracted Plate Number"] != plate_number]

    # Save the cleaned DataFrame back to the CSV file
    df.to_csv(csv_file, index=False)

    print("Recurrent plates and images deleted successfully.")

# Now you can call the function with the CSV file and folder path
csv_file_path = "C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates/ocr_results.csv"
folder_path = "C:/Users/Dell/Desktop/Project/custom_model_lite/custom_model_lite/Detected_Plates"
delete_recurrent_plates_and_images(csv_file_path, folder_path)


Recurrent plates and images deleted successfully.
